# Generate Adjacency Matrix

In [2]:
from DOLPHIN import combine_adj,combine_adj_comp, adj_comp_re_part1, adj_comp_re_part2, adj_comp_re_part3,get_adj_hvg
# #miss one "step02_adj_mat_main_part2_main_2_parallel_run_X"

In [ ]:
metadata = "your_metaData.csv"
df_label = pd.read_csv(metadata, sep='\t')
total_sample_size = len(df_label)

# Define your study name (update this for your specific study)
study_name = "STUDY"

adj_run_num = 100

output_directory = os.path.join(main_folder, "final_data")
graph_directory = os.path.join(main_folder "06_graph_mtx")
exon_gene_featurecount_directory=os.path.join(main_folder, "04_exon_gene_cnt")

# Required files
gene_annotation = "gene_id_annotation.csv"
gtf_pkl_path = "gtf.pkl"
gtf_jun_pkl_path = "df_jun_gtf.pkl"
gtf_path= "Homo_sapiens.GRCh38.107.gtf"

### Combine all Adjacency Matrix

In [ ]:
print("Start Combine Adjacency Matrix")
with tqdm(total=total_sample_size) as pbar_adj:
    for i in range(0, total_sample_size):
        if i%adj_run_num==0:
            pbar_adj=combine_adj(pbar_adj, df_label, graph_directory, gtf_jun_pkl_path, start_idx = i, sample_num=adj_run_num, output_path=output_directory, output_name= study_name)

In [ ]:
### combine all adjacency .h5ad files
total_number_adj_anndata = math.ceil(total_sample_size/adj_run_num)
_anndata_adj_dict = {}
_aa_all_names_lst = []
for j in range(0, total_number_adj_anndata):
    print(j)
    _temp_ad = anndata.read_h5ad(os.path.join(output_directory, "Adjacency_"+study_name+"_"+str(j)+".h5ad"))
    _anndata_adj_dict["AA"+str(j%5)] = _temp_ad

    if j < (len(range(0, total_number_adj_anndata)) // 5) * 5:
        if (j+1)%5 == 0:
            for _i, (_, _ad) in enumerate(_anndata_adj_dict.items()):
                if _i == 0:
                    combine_anndata_adj = _ad
                else:
                    combine_anndata_adj = combine_anndata_adj.concatenate(_ad, index_unique = None, batch_key = None)
            _out_aa_name = "Adjacency_"+study_name+"_"+str(j-4)+"_"+str(j)+".h5ad"
            combine_anndata_adj.write(os.path.join(output_directory,_out_aa_name))
            _anndata_adj_dict = {}
            _aa_all_names_lst.append(_out_aa_name)
    elif j>=(len(range(0, total_number_adj_anndata)) // 5) * 5 and j==max(range(0, total_number_adj_anndata)):
        for _i, (_, _ad) in enumerate(_anndata_adj_dict.items()):
            if _i == 0:
                combine_anndata_adj = _ad
            else:
                combine_anndata_adj = combine_anndata_adj.concatenate(_ad, index_unique = None, batch_key = None)
        _out_aa_name = "Adjacency_"+study_name+"_"+str(total_number_adj_anndata%5+1)+"_"+str(j)+".h5ad" 
        combine_anndata_adj.write(os.path.join(output_directory, _out_aa_name))
        _anndata_adj_dict = {}
        _aa_all_names_lst.append(_out_aa_name)

print(_aa_all_names_lst)
_anndata_aa_all_dict = {}
for _i, _ca_name in enumerate(_aa_all_names_lst):
    _temp_ad = anndata.read_h5ad(os.path.join(output_directory, _ca_name))
    _anndata_aa_all_dict["AA_all"+str(_i)] = _temp_ad
    for _i, (_, _ad) in enumerate(_anndata_aa_all_dict.items()):
        if _i == 0:
            combine_anndata = _ad
        else:
            combine_anndata = combine_anndata.concatenate(_ad, index_unique = None, batch_key = None)
    combine_anndata.write(os.path.join(output_directory,"Adjacency_"+study_name+".h5ad"))


In [ ]:
### Remove All Temporary Datasets
file_pattern = os.path.join(output_directory, f"Adjacency_{study_name}_*")

files_to_remove = glob.glob(file_pattern)

# Remove each file
for file in files_to_remove:
    if os.path.exists(file):  # Check if the file exists
        os.remove(file)
        print(f"Removed: {file}")
    else:
        print(f"File not found: {file}")

### Set Junction to 0 if Both Exon Counts are Zero

In [ ]:
## generate compressed version adjacency matrix
print("Start Generating Compressed version of Adjacency Matrix")
with tqdm(total=total_sample_size) as pbar:
    for i in range(0, total_sample_size):
        if i%adj_run_num==0:
            pbar=combine_adj_comp(pbar, df_label, start_idx = i, sample_num=adj_run_num, output_path=output_directory, output_name= study_name)


In [ ]:
### combine the compressed adjacency matrix
_anndata_ca_dict = {}

total_number_anndata = math.ceil(total_sample_size/adj_run_num)
_ca_all_names_lst = []
print("Combining ", total_number_anndata, " compressed adjacency anndata")
for j in range(0, total_number_anndata):
    print(j)
    _temp_ad = anndata.read_h5ad(os.path.join(output_directory, "AdjacencyComp_"+study_name+"_"+str(j)+".h5ad"))
    _anndata_ca_dict["CA"+str(j%5)] = _temp_ad

    if j < (len(range(0, total_number_anndata)) // 5) * 5:
        if (j+1)%5 == 0:
            for _i, (_, _ad) in enumerate(_anndata_ca_dict.items()):
                if _i == 0:
                    combine_anndata = _ad
                else:
                    combine_anndata = combine_anndata.concatenate(_ad, index_unique = None, batch_key = None)
            _out_ca_name = "AdjacencyComp_"+study_name+"_"+str(j-4)+"_"+str(j)+".h5ad"
            combine_anndata.write(os.path.join(output_directory,_out_ca_name))
            _anndata_ca_dict = {}
            _ca_all_names_lst.append(_out_ca_name)
    elif j>=(len(range(0, total_number_anndata)) // 5) * 5 and j==max(range(0, total_number_anndata)):
        for _i, (_, _ad) in enumerate(_anndata_ca_dict.items()):
            if _i == 0:
                combine_anndata = _ad
            else:
                combine_anndata = combine_anndata.concatenate(_ad, index_unique = None, batch_key = None)
        _out_ca_name = "AdjacencyComp_"+study_name+"_"+str(total_number_anndata%5+1)+"_"+str(j)+".h5ad" 
        combine_anndata.write(os.path.join(output_directory, _out_ca_name))
        _anndata_ca_dict = {}
        _ca_all_names_lst.append(_out_ca_name)

_anndata_ca_all_dict = {}
for _i, _ca_name in enumerate(_ca_all_names_lst):
    _temp_ad = anndata.read_h5ad(os.path.join(output_directory, _ca_name))
    _anndata_ca_all_dict["CA_all"+str(_i)] = _temp_ad
    for _i, (_, _ad) in enumerate(_anndata_ca_all_dict.items()):
        if _i == 0:
            combine_anndata = _ad
        else:
            combine_anndata = combine_anndata.concatenate(_ad, index_unique = None, batch_key = None)
    combine_anndata.write(os.path.join(output_directory,"AdjacencyComp_"+study_name+".h5ad"))


In [ ]:
### Remove All Temporary Datasets
file_pattern = os.path.join(output_directory, f"AdjacencyComp_{study_name}_*")

files_to_remove = glob.glob(file_pattern)

# Remove each file
for file in files_to_remove:
    if os.path.exists(file):  # Check if the file exists
        os.remove(file)
        print(f"Removed: {file}")
    else:
        print(f"File not found: {file}")

### Remove Junctions if All Cells Have a Count of Zero

In [ ]:
adj_comp_re_part1(output_directory, study_name)
adj_comp_re_part2(output_directory, study_name)
adj_comp_re_part3(output_directory, study_name)

### Filter Adjacency Matrix with Highly Variable Genes (HVG)

In [ ]:
get_adj_hvg(output_directory, study_name)